<a href="https://colab.research.google.com/github/rafiqmyura/Learn-Machine-Learning/blob/main/Titanic_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf
print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.8.1


### Load DataSet

In [31]:
train_df = pd.read_csv ("/content/train.csv")
serving_df = pd.read_csv ("/content/test.csv")

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


### Prepare DataSet

In [32]:
def preprocess(df):
  df = df.copy()

  def normalize_name(x):
    return " ". join([v.strip(",()[].\"'") for v in x.split(" ")])

  def ticket_number(x):
    return x.split(" ") [-1]

  def ticket_item(x):
    items = x.split(" ")
    if len(items) == 1:
      return "NONE"
    return "_".join(items[0:-1])

  df["Name"] = df["Name"].apply(normalize_name)
  df["Ticket_number"] = df["Ticket"].apply(ticket_number)
  df["Ticket_item"] = df["Ticket"].apply(ticket_item)
  return df


preprocessed_train_df = preprocess(train_df)
preprocessed_train_serving = preprocess(serving_df)

preprocessed_train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Montvila Rev Juozas,male,27.0,0,0,211536,13.0000,NaN,S,211536,NONE
887,888,1,1,Graham Miss Margaret Edith,female,19.0,0,0,112053,30.0000,B42,S,112053,NONE
888,889,0,3,Johnston Miss Catherine Helen Carrie,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,6607,W./C.
889,890,1,1,Behr Mr Karl Howell,male,26.0,0,0,111369,30.0000,C148,C,111369,NONE


In [33]:
input_features =list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")

print(f"input features: {input_features}")

input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [34]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_df = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_df = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_serving).map(tokenize_names)

### Train model with default parameters


In [37]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Accuracy: 0.8260869383811951 Loss:0.8608942627906799
